In [1]:
import numpy as np

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 7.4 합성곱/풀링 계층 구현하기

## 7.4.1 4차원 배열

In [2]:
# 높이 28, 너비 28, 채널 1, 데이터 10개 
x = np.random.rand(10, 1, 28, 28) # 무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape
x[1].shape

(1, 28, 28)

(1, 28, 28)

In [5]:
# 첫 번째 데이터의 첫 채널의 공간 데이터에 접근하려면..
x[0, 0] # 또는 x[0][0]
x[0, 0].shape

array([[0.37837443, 0.89978782, 0.20430486, 0.0700689 , 0.84873407,
        0.54731344, 0.47152839, 0.37206527, 0.22397149, 0.57505468,
        0.87469221, 0.61831538, 0.73068493, 0.64866651, 0.43195111,
        0.23364392, 0.69964295, 0.32020257, 0.86888942, 0.66426907,
        0.9984468 , 0.8904533 , 0.42434938, 0.0365719 , 0.29964038,
        0.85429526, 0.0554713 , 0.84107897],
       [0.15384228, 0.0038662 , 0.30090161, 0.1149876 , 0.02234798,
        0.01653277, 0.69465171, 0.18763193, 0.45485859, 0.37970995,
        0.43630858, 0.38723536, 0.50120518, 0.44382382, 0.721752  ,
        0.24868234, 0.40912253, 0.75998078, 0.57126032, 0.94721272,
        0.55081118, 0.84520342, 0.73978359, 0.38045316, 0.2499072 ,
        0.40736914, 0.4068049 , 0.86172523],
       [0.93414163, 0.1392412 , 0.97934449, 0.84319038, 0.82612272,
        0.64065101, 0.20681717, 0.74614775, 0.39761172, 0.74881831,
        0.19785016, 0.77451835, 0.59706799, 0.42172205, 0.51140076,
        0.19596881, 0.4808

(28, 28)

## 7.4.3 합성곱 계층 구현하기

In [ ]:
im2col(input_data, filter_h, filter_w, stride=1, pad=0)

In [ ]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) # (데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75)

x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)